## Funciones de utilidad

In [1]:
import sympy as sp
from IPython.display import display, Math

def print_math(sympy_exp, pre_symbol = '', post_symbol = '', convert_pre_post = False):
    # Map some symbols to be reaplaced before printing to keep desired order
    rep_map = {
        'zt': 't',
        'zU_p': 'U_p',
        'z\\dot{U}_p': '\\dot{U}_p',
        'z\\ddot{U}_p': '\\ddot{U}_p',
        'zU': 'U',
        'z\\dot{U}': '\\dot{U}',
        'z\\ddot{U}': '\\ddot{U}',
        'a\\xi': '\\xi',
        'am': 'm',
        'bc': 'c',
    }
    latex_str = sp.latex(sympy_exp)
    if convert_pre_post:
        pre_symbol_latex = sp.latex(pre_symbol)
        post_symbol_latex = sp.latex(post_symbol)
    for old_str in rep_map:
        latex_str = str(latex_str).replace(old_str, rep_map[old_str])
        if pre_symbol != '' and convert_pre_post:
            pre_symbol_latex = pre_symbol_latex.replace(old_str, rep_map[old_str])
        if post_symbol != '' and convert_pre_post:
            post_symbol_latex = post_symbol_latex.replace(old_str, rep_map[old_str])
            
    # if pre_symbol != '' and convert_pre_post:
    #     pre_symbol = pre_symbol_latex
    # if post_symbol != '' and convert_pre_post:
    #     post_symbol = post_symbol_latex
        
    if pre_symbol != '':
        if convert_pre_post:
            pre_symbol = pre_symbol_latex
        latex_str = pre_symbol + ' = ' + latex_str
    if post_symbol != '':
        if convert_pre_post:
            post_symbol = post_symbol_latex
        latex_str = latex_str + ' = ' + post_symbol
    # print(latex_str)
    display(Math(f'{latex_str} \n'))

def print_text(text_str):
    display(Math(f'\\textrm{{{text_str}}}'))

## Encontrando la solución particular

In [2]:
# General symbols
Uu, Uv, Ua, A, B, wn, wd, t, m, k, to, z, c, r, ts = sp.symbols('zU, z\\dot{U}, z\\ddot{U}, A, B, w_n, w_d, zt, am, k, zt_0, a\\xi, bc, r, tr')

# Symbols for the particular solution
Uup, Uvp, Uap, Ap, Bp, Uest = sp.symbols('zU_p, z\\dot{U}_p, z\\ddot{U}_p, A_p, B_p, U_{est}')

# Symbols specific for the loading function
Ome, Fo, a_fac = sp.symbols('\Omega, F_0, a_{fac}')


# Parts of the particular solution and external force
# Sine accel
part_factors = [sp.cos(Ome * t), sp.sin(Ome * t)]
part_cts = [Ap, Bp]
ext_force = Fo * (sp.sin(Ome*t) * sp.cos(Ome*ts) - sp.cos(Ome*t) * sp.sin(Ome*ts))

# Constant
# part_factors = [t, 1]
# part_cts = [Bp, Ap]
# # ext_force = Fo * sp.sin(Ome * ts)
# ext_force = Fo * a_fac

# Free vibration
# part_factors = [1, 1]
# part_cts = [Ap, Bp]
# ext_force = 0

# Linear acceleration (negative slope)
# part_factors = [t, 1]
# part_cts = [Bp, Ap]
# ext_force = Fo * (1 - t/(ts/2))

# Linear acceleration (positive slope)
# part_factors = [t, 1]
# part_cts = [Bp, Ap]
# ext_force = -Fo * (1 - t/(ts/2))

if ext_force != 0:
    # Diferential equation
    print_text('Ecuacion diferencial de gobierno: ')
    ed = m * Ua + c * Uv + k * Uu # - Fo * sp.sin(Ome * t)
    print_math(ed, '', ext_force, True)

    # TODO Divide by m to make things easier and get \xi and wn into play
    ed = sp.expand(ed / m)
    ed = ed.subs(c, 2*z*wn*m)
    ed = ed.subs(k/m, wn**2)
    ext_force = ext_force / m
    print_math(ed, '', ext_force, True)

    # Solution's shape for displacement, velocity and acceleration
    print_text('Forma de la solucion: ')
    Ups = (part_cts[0] * part_factors[0] + part_cts[1] * part_factors[1])
    print_math(Ups, 'U_p',)
    Vps = sp.diff(Ups, t)
    print_math(Vps, '\\dot{U_p}')
    Aps = sp.diff(Vps, t)
    print_math(Aps, '\\ddot{U_p}')

    # Plug solution function into the governing differential equation
    print_text('Reemplazando en la ecuacion diferencial: ')
    ed_rep = ed.subs({Uu: Ups, Uv: Vps, Ua: Aps})
    print_math(ed_rep, '', ext_force, True)

    # TODO Introduce the factor r = Ome/wn
    print_text('Introduciendo factores de simplificacion: ')
    ed_rep = sp.expand(ed_rep / wn**2)
    ed_rep = ed_rep.subs(Ome / wn, r)
    ext_force = sp.expand(ext_force / wn**2)
    ext_force = ext_force.subs(m * wn**2, k)
    ext_force = ext_force.subs(Fo / k, Uest)
    print_math(ed_rep, '', ext_force, True)

    # Get the factors for each part of the function to solve it
    factors_coeff = sp.reduced(ed_rep, part_factors)
    factors_coeff_ext = sp.reduced(ext_force, part_factors)
    # print_text('Factors for the diff. eq.')
    # print_math(ed_rep)
    # print_math(part_factors)
    # print_math(factors_coeff)
    # print_text('Factors for the external force')
    # print_math(ext_force)
    # print_math(part_factors)
    # print_math(factors_coeff_ext)

    # Create the system of equations
    eqns = []
    for i in range(len(factors_coeff[0])):
        eqns.append(factors_coeff[0][i] - factors_coeff_ext[0][i])

    # Solve the system of equations
    sol = sp.solve(eqns, part_cts)

    for cnt in part_cts:
        if not cnt in sol:
            sol[cnt] = 0

    print_text('Coeficientes encontrados: ')
    for res in sol:
        sol[res] = sp.simplify(sp.factor(sol[res]))
        print_math(sol[res], res, '', True)

    print_text('Solución particular: ')
    # Plug found coefficients into the governing equation
    Ups = sp.simplify(Ups.subs(sol))
    Vps = sp.simplify(Vps.subs(sol))
    Aps = sp.simplify(Aps.subs(sol))
else:
    Ups = 0
    Vps = 0
    Aps = 0

print_math(Ups, 'U_p',)
print_math(Vps, '\\dot{U}_p',)
print_math(Aps, '\\ddot{U}_p',)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Solución total con $U = U_p + U_h$

In [3]:
Uuo, Uvo, Uao = sp.symbols('zU_0, \\dot{U}_0, \\ddot{U}_0')

# ----------- Damped free vibration -----------
# Diferential equation
print_text('Ecuacion diferencial de gobierno: ')
ed = m * Ua + c * Uv + k * Uu
print_math(ed, '', '0')
print_text('Forma de la solucion: ')
Us = (A * sp.cos(wd * t) + B * sp.sin(wd * t)) * sp.exp(-z*wn*t) + Ups
# Us = (A * sp.cos(wd * t) + B * sp.sin(wd * t)) * sp.exp(-z*wn*t)
print_math(Us, 'U')
Vs = sp.diff(Us, t)
print_math(Vs, '\\dot{U}')
As = sp.diff(Vs, t)
print_math(As, '\\ddot{U}')

print_text('Dadas las condiciones de borde, encontrar las condiciones de borde: ')
Uso = Us - Uuo
Uso = Uso.subs(t, to)
print_math(Uso, '', '0')
Uso = Uso.subs(to, 0)
Vso = Vs - Uvo
Vso = Vso.subs(t, to)
print_math(Vso, '', '0')
Vso = Vso.subs(to, 0)

print_text('Sistema de ecuaciones a resolver: ')
print_math(Uso, '', '0')
print_math(Vso, '', '0')


print_text('Coeficientes encontrados: ')
coeff_sol = sp.solve([Uso, Vso], [A, B])
for coeff in coeff_sol:
    coeff_sol[coeff] = sp.simplify(sp.factor(coeff_sol[coeff]))
    print_math(coeff_sol[coeff], sp.latex(coeff))

print_text('Soluciones finales:')
Uu = sp.simplify(Us.subs(coeff_sol))
print_math(Uu, 'U')
Uv = sp.simplify(Vs.subs(coeff_sol))
print_math(Uv, '\\dot{U}')
Ua = sp.simplify(As.subs(coeff_sol))
print_math(Ua, '\\ddot{U}')


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Traduciendo las expresiones a Matlab

In [4]:
U_str = str(Uu)
V_str = str(Uv)
A_str = str(Ua)

U_str = U_str.replace('**', '^')
V_str = V_str.replace('**', '^')
A_str = A_str.replace('**', '^')
used_symbols =  Uu.atoms(sp.Symbol)

rep_map = {
    'w_n': 'wn',
    '\\Omega' : 'Ome',
    'zU_0' : 'Uo',
    'r' : 'r',
    'w_d' : 'wd',
    'zt' : 't',
    'U_{est}' : 'Uest',
    '\\dot{U}_0' : 'Vo',
    'a\\xi' : 'z',
    'tr' : 'offset',
    'a_{fac}' : 'a_fac',
}

for sym in used_symbols:
    U_str = U_str.replace(str(sym), rep_map[str(sym)])
    V_str = V_str.replace(str(sym), rep_map[str(sym)])
    A_str = A_str.replace(str(sym), rep_map[str(sym)])

print('Expresión Matlab (desplazamiento): ')
print(U_str)

print('Expresión Matlab (velocidad): ')
print(V_str)

print('Expresión Matlab (aceleracion): ')
print(A_str)


Expresión Matlab (desplazamiento): 
(-Uest*wd*(2*z*r*cos(Ome*(offset - t)) - r^2*sin(Ome*(offset - t)) + sin(Ome*(offset - t)))*exp(z*wn*t) + wd*(2*Uest*z*r*cos(Ome*offset) - Uest*r^2*sin(Ome*offset) + Uest*sin(Ome*offset) + 4*z^2*r^2*Uo + r^4*Uo - 2*r^2*Uo + Uo)*cos(wd*t) + (2*Uest*Ome*z*r*sin(Ome*offset) + Uest*Ome*r^2*cos(Ome*offset) - Uest*Ome*cos(Ome*offset) + 2*Uest*z^2*r*wn*cos(Ome*offset) - Uest*z*r^2*wn*sin(Ome*offset) + Uest*z*wn*sin(Ome*offset) + 4*Vo*z^2*r^2 + Vo*r^4 - 2*Vo*r^2 + Vo + 4*z^3*r^2*wn*Uo + z*r^4*wn*Uo - 2*z*r^2*wn*Uo + z*wn*Uo)*sin(wd*t))*exp(-z*wn*t)/(wd*(4*z^2*r^2 + r^4 - 2*r^2 + 1))
Expresión Matlab (velocidad): 
(-Uest*Ome*wd*(2*z*r*sin(Ome*(offset - t)) + r^2*cos(Ome*(offset - t)) - cos(Ome*(offset - t)))*exp(z*wn*t) - z*wn*(wd*(2*Uest*z*r*cos(Ome*offset) - Uest*r^2*sin(Ome*offset) + Uest*sin(Ome*offset) + 4*z^2*r^2*Uo + r^4*Uo - 2*r^2*Uo + Uo)*cos(wd*t) + (2*Uest*Ome*z*r*sin(Ome*offset) + Uest*Ome*r^2*cos(Ome*offset) - Uest*Ome*cos(Ome*offset) + 2*Uest*z^